In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

DATA_DIR = '/content/drive/My Drive/proto_data/'
PICKLED_DIR = os.path.join(DATA_DIR, 'pickled/')
#CONLLU_DIR = os.path.join(DATA_DIR, 'WSJ_conllus/')
#MODEL_DIR = '../saved_models/'

PROTO_TSV = os.path.join(DATA_DIR, 'protoroles_eng_pb_08302015.tsv')
#GLOVE_FILE = {'100': os.path.join(DATA_DIR, 'glove.6B.100d.txt') }

SPLITS = ['train', 'dev', 'test'] 

PROPERTIES = ['instigation', 'volition', 'awareness', 'sentient',
'exists_as_physical', 'existed_before', 'existed_during', 'existed_after',
'created', 'destroyed', 'predicate_changed_argument', 'change_of_state', 
'changes_possession', 'change_of_location', 'stationary', 'location_of_event', 
'makes_physical_contact', 'manipulated_by_another']

PAD_TOKEN = '<pad>'
UNK_TOKEN = '<unk>'


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive/proto_modules')

In [4]:
from collections import defaultdict
import os
import pickle
from tqdm import tqdm
import random
import math

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device: ', device)

#import data_utils

device:  cuda:0


In [0]:
from torch import unsqueeze
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, \
        pack_padded_sequence, pad_sequence

class LSTM(nn.Module):
    def __init__(self,
            vocab_size=None,
            emb_size=None,
            h_size=None,
            shared_size=None,
            padding_idx=None,
            emb_np=None,
            properties=None):
        super(LSTM, self).__init__()

        self.n_properties=len(properties)

        self.word_emb = nn.Embedding(
                vocab_size,
                emb_size,
                padding_idx=padding_idx)
        self.word_emb.weight.data.copy_(torch.Tensor(emb_np))
        self.word_emb.weight.requires_grad = False
        
        directions = 2
        self.lstm = nn.LSTM(
                input_size=emb_size,    
                hidden_size=h_size,
                num_layers=1,
                bidirectional=(directions == 2),
                batch_first=True,
                dropout=0.1,
                bias=True)
        
        self.lstm_drop = nn.Dropout(0.)
        
        concatenated_embs_size = 2*(directions*h_size) # If using LSTM
        #concatenated_embs_size = (2*emb_size) # If using DAN
        shared_size=concatenated_embs_size
        specific_size = int(shared_size / 2)

        # IF using LSTM
        self.shared = nn.Sequential(
            nn.Linear(concatenated_embs_size, shared_size, bias=True),
        #    nn.Dropout(0.2),
            nn.ReLU())
        
        # DAN
        # self.shared = nn.Sequential(
        #     nn.Linear(concatenated_embs_size, shared_size, bias=True),
        #     nn.Dropout(0.3),
        #     nn.ReLU(),
        #     nn.Linear(shared_size, shared_size, bias=True),
        #     nn.Dropout(0.3),
        #     nn.ReLU(),
        #     nn.Linear(shared_size, shared_size, bias=True),
        #     nn.Dropout(0.3),
        #     nn.ReLU())

        # Attention-based
        self.shared = nn.Sequential(
            nn.Linear(self.n_properties*concatenated_embs_size, self.n_properties * specific_size, bias=True),
        #    nn.Dropout(0.2),
            nn.ReLU())
        


        # Output is (B, shared_size)

        #self.mlps = {p: nn.Linear(2 * h_size, 2) for p in PROPERTIES}
        self.mlp = nn.Linear(shared_size*self.n_properties, self.n_properties, bias=True)


    def forward(self, sents, sent_lens, preds, heads):

        # Sort the sentences so that the LSTM can process properly
        lens_sorted = sent_lens
        words_sorted = sents
        indices = None
        if(len(sents) > 1):
            lens_sorted, indices = torch.sort(lens_sorted, descending=True)
            lens_sorted = lens_sorted.to(device)
            indices = indices.to(device)
            words_sorted = words_sorted.index_select(0, indices).to(device)

        w_embs = self.word_emb(words_sorted)

        packed_lstm_input = pack_padded_sequence(
                w_embs, lens_sorted, batch_first=True)

        outputs, _ = self.lstm(packed_lstm_input)
        outputs, _ = pad_packed_sequence(outputs, batch_first=True)

        # Unsort sentences to return to proper alignment with labels
        if len(outputs) > 1:
            outputs = unsort(outputs, indices)
          
        outputs = self.lstm_drop(outputs)

        # outputs : (B, L, h_size), assuming preds (B)
        B, L, h_size = outputs.shape
        pred_reps = outputs[np.arange(B), preds] # expecting (B, h_size)
        head_reps = outputs[np.arange(B), heads] # same as above

        # DAN version
        # outputs = self.word_emb(sents)
        # B, L, h_size = outputs.shape
        # pred_reps = outputs[np.arange(B), preds] # expecting (B, h_size)
        # head_reps = outputs[np.arange(B), heads] # same as above

        # Get pred-arg representation (same for DAN and LSTM)
        shared_input = torch.cat([pred_reps, head_reps], dim=-1) # (B, 2*h_size)

        #mlp_input = self.shared(shared_input) # (B, size_shared) # For lstm
        #mlp_input = shared_input # (B, size_shared)
        mlp_input = self.shared(shared_input.repeat(1, self.n_properties)) # (B, n_properties * size_specific) # For attn

        reshaped = mlp_input.view(B, self.n_properties, -1)
        print(f'reshaped shape {reshaped.shape}')

        attn_scores = torch.bmm(reshaped, torch.transpose(reshaped, 2, 1))
        #print(f'Attnscores shape {attn_scores.shape}')
        dists = F.softmax(attn_scores, -1)
        #print(f'dists shape {dists.shape}')
        # sums = dists.sum(-1)
        # print(sums.shape)
        # print(sums.sum())

        # Zero out self-attn score? Prob equiv to zeroing + adding original vector back

        # Now get attention-weighted sum for each prop -> (B, n_properties, specific_size)

        # Then choose: concatenate or sum orig & attn-weighted sum

        # Finally, 
        breakpoint()

        #logits = {p: None for p in PROPERTIES}
        #for p in PROPERTIES:
        #    logits[p] = self.mlps[p](mlp_input)

        mlp_input = mlp_input.repeat(1, self.n_properties) # Don't repeat on axis 0, 
        logits = self.mlp(mlp_input) # (B, properties)

        return logits


        
    def predict(self, sents, sent_lens, preds, heads):

        logits = self.forward(sents, sent_lens, preds, heads)
        predictions = (logits.sign() + 1) / 2
        #preds = F.sigmoid(logits)
        #preds = argmax(logits, -1) # assuming logits a (num_props X 2) array

        return predictions
 
def unsort(batch, indices):
    indices_inverted = torch.argsort(indices)
    batch = batch.index_select(0, indices_inverted)
    return batch

In [0]:
def train_lstm(args, model, X, y):
    epochs = args['epochs']
    batch_size = args['batch_size']
    lr = args['lr']


    # Data loaders
    loader_train = data_loader(X['train'], y['train'],
            batch_size=batch_size, shuffle_idx=True)
    loader_dev = data_loader(X['dev'], y['dev'],
            batch_size=batch_size, shuffle_idx=False)
    n_train_batches = math.ceil(len(X['train']) / batch_size)
    n_dev_batches = math.ceil(len(X['dev']) / batch_size)

    # Optimizer
    opt = Adam(model.parameters(), lr=lr, betas=[0.9, 0.999])

    # Train loop
    try:
        for e in range(epochs):
            model.train()
            for b in tqdm(
                    range(n_train_batches), 
                    ascii=True, 
                    desc=f'Epoch {e+1}/{epochs} progress', 
                    ncols=80):
                opt.zero_grad()
                sents, sent_lens, preds, heads, labels = next(loader_train)
                logits = model(sents, sent_lens, preds, heads)
                loss = bce_loss(logits, labels)
                loss.backward()
                opt.step()
        
            with torch.no_grad():
              model.eval()
              tp = 0
              fp = 0
              fn = 0
              for b in tqdm(range(n_dev_batches), ascii=True, desc=f'Evaluating progress', ncols=80):
                sents, sent_lens, preds, heads, labels = next(loader_dev)
                results_, metrics_ = evaluate(args, model, sents, sent_lens, preds, heads, labels)
                tp += results_['tp']
                fp += results_['fp']
                fn += results_['fn']
              
              F, precision, recall = F_precision_recall(tp, fp, fn)

              print(f"Epoch {e}, F={F*100:.2f}, p={precision*100:.2f}, r={recall*100:.2f}")

    except KeyboardInterrupt:
        pass
    # End of train loop

    # Eval time!
    model.eval()
    with torch.no_grad():
      tp = 0
      fp = 0
      fn = 0
      for b in tqdm(range(n_dev_batches), ascii=True, desc=f'Evaluating progress', ncols=80):
        sents, sent_lens, preds, heads, labels = next(loader_dev)
        results_, metrics_ = evaluate(args, model, sents, sent_lens, preds, heads, labels)
        tp += results_['tp']
        fp += results_['fp']
        fn += results_['fn']
      
      F, precision, recall = F_precision_recall(tp, fp, fn)

      print(f"F={F*100:.2f}, p={precision*100:.2f}, r={recall*100:.2f}")
    
    return


def bce_loss(logits, labels):
    # Expected labels : (B, num_properties)
    loss = F.binary_cross_entropy_with_logits(logits, labels)
    return loss


def data_loader(X, y, batch_size=None, shuffle_idx=False):
    data = list(zip(X, y))
    idx = list(range(len(data)))
    while True:
        if shuffle_idx:
            random.shuffle(idx) # In-place shuffle
        
        for span in idx_spans(idx, batch_size):
            batch = [data[i] for i in span]
            yield prepare_batch(batch)


def idx_spans(idx, span_size):
    for i in range(0, len(idx), span_size):
        yield idx[i:i+span_size]


def prepare_batch(batch):
    # batch[i] = X, y
    batch_size = len(batch)
    sent_lens = torch.LongTensor([len(x[0][0]) for x in batch])
    max_length = torch.max(sent_lens).item()
    n_properties = len(batch[0][1])

    # Zero is padding index
    sents = torch.zeros((batch_size, max_length)).long().to(device)
    preds = torch.zeros(batch_size).long().to(device)
    heads = torch.zeros(batch_size).long().to(device)
    labels = torch.zeros(batch_size, n_properties).to(device)

    for i, (X_batch, y_batch) in enumerate(batch):
        sent, (pred_idx, head_idx) = X_batch
        sents[i,:len(sent)] = torch.LongTensor(sent)
        preds[i] = pred_idx
        heads[i] = head_idx
        labels[i] = torch.tensor(y_batch)

    return sents, sent_lens, preds, heads, labels

In [0]:
def evaluate(args, model, sents, sent_lens, preds, heads, labels):
    # Get predictions
    predictions = model.predict(sents, sent_lens, preds, heads)

    predictions, labels = predictions.cpu().numpy(), labels.cpu().numpy()
    n_correct = (predictions == labels).astype(int).sum()

    # Precision, Recall
    eq = predictions == labels
    neq = predictions != labels

    pos_preds = predictions == 1
    neg_preds = predictions == 0

    tp = np.where(pos_preds, eq, 0).astype(int).sum()
    fp = np.where(pos_preds, neq, 0).astype(int).sum()
    fn = np.where(neg_preds, neq, 0).astype(int).sum()
    
    results = {
            'tp': tp,
            'fp': fp,
            'fn': fn
            }

    F, precision, recall = F_precision_recall(tp, fp, fn)
    metrics = {'F': F, 'precision': precision, 'recall': recall}

    return results, metrics


def F_precision_recall(tp, fp, fn):
    if tp + fp > 0.:
        precision = tp / (tp + fp)
    else:
        precision = 0.

    if tp + fn > 0.:
        recall = tp / (tp + fn)
    else:
        recall = 0.

    if precision + recall > 0.:
        F = (2 * precision * recall) / (precision + recall)
    else:
        F = 0.

    return F, precision, recall


def micro_average(results):
    tp, fp, fn = 0, 0, 0
    for v in results.values():
        tp += v['tp']
        fp += v['fp']
        fn += v['fn']
    
    return F_precision_recall(tp, fp, fn)


In [0]:
def get_data(args):
    df = pd.read_csv(PROTO_TSV, sep='\t')

    # Sentences
    sent_ids = set(df['Sentence.ID'].tolist())
    sents_path = os.path.join(PICKLED_DIR, 'sents.pkl')
    sents = None
    with open(sents_path, 'rb') as f:
      sents = pickle.load(f)

    # Dependency data
    # dependencies_path = os.path.join(PICKLED_DIR, 'dependencies.pkl')
    # with open(dependencies_path, 'rb') as f:
    #     deps, deps_just_tokens = pickle.load(f)  
    # sents['dependencies'] = deps
    # sents['deps_just_tokens'] = deps_just_tokens


    # Instances
    path = os.path.join(PICKLED_DIR, 'instances.pkl')
    proto_instances = None
    possible = None # Data to compare to SPRL paper
    with open(path, 'rb') as f:
      proto_instances, possible = pickle.load(f)

    # Word embedding data
    w2e = None
    path = os.path.join(PICKLED_DIR, f"glove_{args['glove_d']}.pkl")
    with open(path, 'rb') as f:
      w2e = pickle.load(f)

    w2i, i2w = None, None
    emb_np = None
    X, y = None, None
    dicts_path = os.path.join(PICKLED_DIR, 'dicts.pkl')
    with open(dicts_path, 'rb') as f:
        w2i, i2w = pickle.load(f)
    
    emb_np_path = os.path.join(PICKLED_DIR, 'emb_np.pkl')
    with open(emb_np_path, 'rb') as f:
        emb_np = pickle.load(f)
    
    lstm_data_path = os.path.join(PICKLED_DIR, 'lstm_data.pkl')
    with open(lstm_data_path, 'rb') as f:
        X, y = pickle.load(f)

    return {'df': df, 
            'proto_instances': proto_instances, 
            'possible': possible,
            'sents': sents,
            'w2e': w2e,
            'sent_ids': sent_ids,
            'lstm_data': (X,y),
            'dicts': (w2i, i2w),
            'emb_np': emb_np}


In [86]:
args = {
    'epochs': 20,
    'seed': 7,
    'lr': 1e-3,
    'batch_size': 100,
    'h_size': 300,
    #'shared_size': 300,
    'glove_d': 300
}

seed = args['seed']

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

data = get_data(args)

w2i, i2w = data['dicts']
emb_np = data['emb_np']
X, y = data['lstm_data']

model = LSTM(
                vocab_size=len(w2i),
                emb_size=int(args['glove_d']),
                h_size=args['h_size'],
                #shared_size=args['shared_size'],
                padding_idx=w2i[PAD_TOKEN],
                emb_np=emb_np,
                properties=PROPERTIES)
model.to(device)

train_lstm(args, model, X, y)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Epoch 1/20 progress:   0%|                               | 0/78 [00:00<?, ?it/s]

reshaped shape torch.Size([100, 18, 600])
Attnscores shape torch.Size([100, 18, 18])
dists shape torch.Size([100, 18, 18])
torch.Size([100, 18])
tensor(1800., device='cuda:0', grad_fn=<SumBackward0>)


NameError: ignored